## Descripcion

## Extraccion

In [94]:
import pandas as pd
import numpy as np

In [2]:
# Titles
Amazon = pd.read_csv("data/amazon_prime_titles.csv")
Disney = pd.read_csv("data/disney_plus_titles.csv")
Hulu = pd.read_csv("data/hulu_titles.csv")
Netflix = pd.read_csv("data/netflix_titles.csv")

In [30]:
# Scores
a = pd.read_csv("data/ratings/1.csv")
b = pd.read_csv("data/ratings/2.csv")
c = pd.read_csv("data/ratings/3.csv")
d = pd.read_csv("data/ratings/4.csv")
e = pd.read_csv("data/ratings/5.csv")
f = pd.read_csv("data/ratings/6.csv")
g = pd.read_csv("data/ratings/7.csv")
h = pd.read_csv("data/ratings/8.csv")


In [3]:
print('Amazon')
print('Tiene duplicados:',Amazon.duplicated().any())
print('Filas, Columnas:',Amazon.shape)
print(Amazon.isnull().sum())
Amazon.tail(3)

Amazon
Tiene duplicados: False
Filas, Columnas: (9668, 12)
show_id            0
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou..."
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ..."
9667,s9668,Movie,Harry Brown,Daniel Barber,"Michael Caine, Emily Mortimer, Joseph Gilgun, ...",NaN,NaN,2010,R,103 min,"Action, Drama, Suspense","Harry Brown, starring two-time Academy Award w..."


In [4]:
print('Disney')
print('Tiene duplicados:',Disney.duplicated().any())
print('Filas, Columnas:',Disney.shape)
print(Disney.isnull().sum())
Disney.tail(3)

Disney
Tiene duplicados: False
Filas, Columnas: (1450, 12)
show_id           0
type              0
title             0
director        473
cast            190
country         219
date_added        3
release_year      0
rating            3
duration          0
listed_in         0
description       0
dtype: int64


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1447,s1448,Movie,Eddie the Eagle,Dexter Fletcher,"Tom Costello, Jo Hartley, Keith Allen, Dickon ...","United Kingdom, Germany, United States","December 18, 2020",2016,PG-13,107 min,"Biographical, Comedy, Drama","True story of Eddie Edwards, a British ski-jum..."
1448,s1449,Movie,Bend It Like Beckham,Gurinder Chadha,"Parminder Nagra, Keira Knightley, Jonathan Rhy...","United Kingdom, Germany, United States","September 18, 2020",2003,PG-13,112 min,"Buddy, Comedy, Coming of Age",Despite the wishes of their traditional famili...
1449,s1450,Movie,Captain Sparky vs. The Flying Saucers,Mark Waring,Charlie Tahan,United States,"April 1, 2020",2012,TV-G,2 min,"Action-Adventure, Animals & Nature, Animation",View one of Sparky's favorite home movies.


In [5]:
print('Hulu')
print('Tiene duplicados:',Hulu.duplicated().any())
print('Filas, Columnas:',Hulu.shape)
print(Hulu.isnull().sum())
Hulu.tail(3)

Hulu
Tiene duplicados: False
Filas, Columnas: (3073, 12)
show_id            0
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating           520
duration         479
listed_in          0
description        4
dtype: int64


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
3070,s3071,TV Show,The Fades,NaN,NaN,United Kingdom,NaN,2011,TV-14,1 Season,"Horror, International, Science Fiction",Seventeen-year-old Paul is haunted by apocalyp...
3071,s3072,TV Show,The Twilight Zone,NaN,NaN,United States,NaN,1959,TV-PG,5 Seasons,"Classics, Science Fiction, Thriller",Rod Serling's seminal anthology series focused...
3072,s3073,TV Show,Tokyo Magnitude 8.0,NaN,NaN,Japan,NaN,2009,TV-14,1 Season,"Anime, Drama, International",The devastation is unleashed in the span of se...


In [6]:
print('Netflix')
print('Tiene duplicados:',Netflix.duplicated().any())
print('Filas, Columnas:',Netflix.shape)
print(Netflix.isnull().sum())
Netflix.tail(3)

Netflix
Tiene duplicados: False
Filas, Columnas: (8807, 12)
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
8806,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


## Transformacion

In [7]:
# Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, 
# seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)
id_A = 'a' + Amazon['show_id']
Amazon.insert(0, 'id', id_A)
Amazon.insert(1, 'platform', 'amazon')

id_D = 'd' + Disney['show_id']
Disney.insert(0, 'id', id_D)
Disney.insert(1, 'platform', 'disney')

id_H = 'h' + Hulu['show_id']
Hulu.insert(0, 'id', id_H)
Hulu.insert(1, 'platform', 'hulu')

id_N = 'n' + Netflix['show_id']
Netflix.insert(0, 'id', id_N)
Netflix.insert(1, 'platform', 'netflix')

In [8]:
All = pd.concat([Amazon,Disney,Hulu,Netflix], axis=0).reset_index()
print('All platforms')
print('Files, Columns:',All.shape)
print(All.isnull().sum())
All.tail(3)


All platforms
Files, Columns: (22998, 15)
index               0
id                  0
platform            0
show_id             0
type                0
title               0
director         8259
cast             5321
country         11499
date_added       9554
release_year        0
rating            864
duration          482
listed_in           0
description         4
dtype: int64


,index,id,platform,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
22995,8804,ns8805,netflix,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
22996,8805,ns8806,netflix,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
22997,8806,ns8807,netflix,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


In [9]:
# Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”)
All['rating'].fillna('G',inplace=True)
print(All['rating'].isnull().sum())

0


In [10]:
# De haber fechas, deberán tener el formato AAAA-mm-dd
All['date_added'] = pd.to_datetime(All['date_added'])
All.tail(3)

,index,id,platform,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
22995,8804,ns8805,netflix,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
22996,8805,ns8806,netflix,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
22997,8806,ns8807,netflix,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,2019-03-02,2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


In [11]:
# Los campos de texto deberán estar en minúsculas, sin excepciones
lowercase = lambda s:s.lower() if type(s) == str else s
All = All.applymap(lowercase)

In [12]:
# El campo duration debe convertirse en dos campos: duration_int y duration_type. 
# El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)
All[['duration_int','duration_type']] = All['duration'].str.split(' ', expand=True)
All.drop(columns=['duration'], inplace=True)
headers = list(All.columns)
print(headers)          # Print headers

['index', 'id', 'platform', 'show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'listed_in', 'description', 'duration_int', 'duration_type']


In [13]:
All.nunique()

index             9668
id               22998
platform             4
show_id           9668
type                 2
title            22042
director         10095
cast             16744
country            886
date_added        2003
release_year       101
rating             105
listed_in         1687
description      22669
duration_int       225
duration_type        3
dtype: int64

In [14]:
All = All.reindex(columns=['id', 'platform', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'duration_int', 'duration_type', 'listed_in', 'description'])
All.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             22998 non-null  object        
 1   platform       22998 non-null  object        
 2   type           22998 non-null  object        
 3   title          22998 non-null  object        
 4   director       14739 non-null  object        
 5   cast           17677 non-null  object        
 6   country        11499 non-null  object        
 7   date_added     13444 non-null  datetime64[ns]
 8   release_year   22998 non-null  int64         
 9   duration_int   22516 non-null  object        
 10  duration_type  22516 non-null  object        
 11  listed_in      22998 non-null  object        
 12  description    22994 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(11)
memory usage: 2.3+ MB


## Adecuacion

In [15]:
All.isnull().sum()

id                   0
platform             0
type                 0
title                0
director          8259
cast              5321
country          11499
date_added        9554
release_year         0
duration_int       482
duration_type      482
listed_in            0
description          4
dtype: int64

In [16]:
All['duration_int'] = All['duration_int'].fillna(0)
All['duration_int'] = All['duration_int'].astype(int)
All['release_year'] = All['release_year'].astype(int) 
All.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             22998 non-null  object        
 1   platform       22998 non-null  object        
 2   type           22998 non-null  object        
 3   title          22998 non-null  object        
 4   director       14739 non-null  object        
 5   cast           17677 non-null  object        
 6   country        11499 non-null  object        
 7   date_added     13444 non-null  datetime64[ns]
 8   release_year   22998 non-null  int64         
 9   duration_int   22998 non-null  int64         
 10  duration_type  22516 non-null  object        
 11  listed_in      22998 non-null  object        
 12  description    22994 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 2.3+ MB


In [25]:
All.nunique()

id               22998
platform             4
type                 2
title            22042
director         10095
cast             16744
country            886
date_added        2003
release_year       101
duration_int       225
duration_type        3
listed_in         1687
description      22669
dtype: int64

In [ ]:
import collections
collections.Counter(All['duration_type'])

Counter({'min': 15999, 'season': 4183, 'seasons': 2334, nan: 482})

In [28]:
All['duration_type'] = All['duration_type'].replace('seasons','season')
collections.Counter(All['duration_type'])

Counter({'min': 15999, 'season': 6517, nan: 482})

## Transformacion Score

In [85]:
Score = pd.concat([a,b,c,d,e,f,g,h], axis=0).reset_index()
print('Score')
print('Files, Columns:',Score.shape)
# print(Score.isnull().sum())
Score.tail(3)

Score
Files, Columns: (11024289, 5)
index        0
userId       0
rating       0
timestamp    0
movieId      0
dtype: int64


,index,userId,rating,timestamp,movieId
11024286,1499997,124380,3.5,1196785679,hs305
11024287,1499998,124380,4.5,1196787089,ns7881
11024288,1499999,124380,1.5,1196785847,as883


In [86]:
Score = Score.rename(columns={'rating':'score'})
Score['year'] = pd.to_datetime(Score['timestamp'], unit='s').dt.year
Score['platf'] = Score['movieId'].str[0]
Score = Score[['platf','score','year','movieId']]
Score.tail(3)

,platf,score,year,movieId
11024286,h,3.5,2007,hs305
11024287,n,4.5,2007,ns7881
11024288,a,1.5,2007,as883


In [87]:
# Score['score'].mean()
pd.Series({'score': Score['score'].mean()})

score    3.533455
dtype: float64

In [92]:
# Definir una función de agregación que calcule el promedio del score
# def rating_avg(Score):
#     return pd.Series({'score': Score['score'].mean()})

# Agrupar los datos por movieID y aplicar la función de agregación
# Score.groupby('movieId').agg(rating_avg)
# Score.groupby('movieId').agg(pd.Series({'score': Score['score'].mean()}))
S = Score.groupby('movieId').agg(pd.Series({'score': 'mean'}))
S.describe()

/var/folders/qr/g6x89n_d48j570j5qt_7mrhr0000gn/T/ipykernel_70695/3342074180.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  S = Score.groupby('movieId').agg(pd.Series({'score': 'mean'}))


,score
count,22998.000000
mean,3.533443
std,0.048564
min,3.336478
25%,3.500000
50%,3.533673
75%,3.567000
max,3.724512


In [ ]:
# def weighted_mean(group):
#     scores = group["score"]
#     weights = group["rating_count"]
#     return np.average(scores, weights=weights)

# Score.groupby("movieId").agg(weighted_mean)

In [ ]:
# import numpy as np

# def weighted_mean(df):
#     return np.average(df['score'], weights=df['weight'])

# # Score.groupby('movieId').agg({'score': weighted_mean(Score)})
# Score.groupby('movieId').agg({'score': lambda x: weighted_mean(x)})


In [55]:
# Función para obtener la inicial de un nombre
# def get_initial(movieId:str):
#     if movieId[0] == 'a':
#         platform = 'amazon'
#     elif movieId[0] == 'd':
#         platform = 'disney'
#     elif movieId[0] == 'h':
#         platform = 'hulu'
#     elif movieId[0] == 'n':
#         platform = 'netflix'
#     else: platform = '' 
    
#     return platform

# get_initial('as883')

# Aplicar la función a la columna 'nombre' para crear la columna 'inicial'
# df['inicial'] = df['nombre'].apply(get_initial)

'amazon'

## Carga

In [29]:
All.to_csv('data/all.csv', index=False)

In [82]:
Score.to_csv('data/score.csv', index=False)

In [107]:
S.to_csv('data/score_mean.csv', index=False)